<a href="https://colab.research.google.com/github/Andrewsas/trabalho-fundamentos-cid/blob/master/%20Modulo_4/Projeto_Final_Modulo_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>